
## Age groups, notations and data source

The population is stratified into $G = 8$ age groups with lower bounds

$$
\mathcal{A} = \{0, 3, 5, 10, 15, 18, 40, 65\}.
$$

While the final age group is open-ended to capture all individuals aged 65 years and above in our model, age is assumed to be capped at 120 years for the purpose of implementing the social mixing methods below.
Let $i,j \in \{1,\dots,G\}$ index age groups, and let $[a_i, b_i]$ denote the range of single ages belonging to age group $i$.

Calendar time is denoted by $t$ (in years). We use data and projections from the UN Population Prospects to inform age-specific population sizes (available for 1950–2100) and age-specific fertility rates (1950–2023). For years outside the available data range, all time-dependent quantities are clamped to the nearest available year.

---

## Overview of the mixing matrix

At each time $t$, the model constructs an age-specific contact matrix

$$
\mathbf{C}(t) = \left(C_{ij}(t)\right)_{i,j=1}^G,
$$

where $C_{ij}(t)$ represents the per-capita rate at which individuals in age group $j$ make potentially infectious contacts with individuals in age group $i$ at time $t$.

The construction proceeds in three steps:

1. construction of a symmetric per-pair contact intensity matrix $\mathbf{S}(t)$;
2. scaling by the population size of the contacted age group;
3. normalisation by the spectral radius.

The term $S_{ij}(t)$ of the symmetric contact matrix represents the rate of contact per unit time at time $t$ between two typical individuals, one aged i and the other aged j. $\mathbf{S}(t)$ is defined as a weighted sum of three components corresponding to uniform background mixing, assortative mixing, and parent–child mixing patterns. These components are detailed in the following sections.

Three parameters governing social mixing are estimated during model calibration using a Metropolis algorithm:

- background mixing strength $\beta_{\mathrm{bg}}$;
- assortative age spread $a_{\mathrm{spread}}$;
- parent–child mixing strength $\kappa_{\mathrm{pc}}$.

---

## Population weights within age groups
The computation of the assortative and parent–child mixing patterns described below involves summing over single ages within each model age group. However, to account for the possibility that the age distribution within an age group is not uniform—particularly for wider age ranges—we define population weights at the individual-age level.

Let $N(a,t)$ denote the population size at single age $a$ and time $t$.  
For each age group $i$, the relative population weight of age $a \in [a_i,b_i]$ is defined as

$$
w_i(a,t) =
\frac{N(a,t)}{\sum_{a' \in [a_i,b_i]} N(a',t)},
$$

so that $\sum_{a \in [a_i,b_i]} w_i(a,t) = 1$.  

---

## Symmetric per-pair contact intensity

For each pair of age groups $i$ and $j$, the symmetric per-pair contact intensity is defined as

$$
S_{ij}(t) =
\beta_{\mathrm{bg}}
+ S^{\mathrm{assort}}_{ij}(t)
+ \kappa_{\mathrm{pc}} \, S^{\mathrm{pc}}_{ij}(t),
$$

where $\beta_{\mathrm{bg}}$ is a calibrated scalar defining the level of background (uniform) mixing, and $S^{\mathrm{assort}}_{ij}(t)$ and $S^{\mathrm{pc}}_{ij}(t)$ denote the assortative and parent-child components, respectively. The parameter $\kappa_{\mathrm{pc}}$ is an additional calibrated scalar that modulates the relative contribution of parent-child interactions to overall mixing.

Matrix symmetry is enforced by setting $S_{ij}(t) = S_{ji}(t) \quad \forall i,j,t$.

### Assortative age mixing

Assortative mixing is modelled as an exponentially decaying function of age difference:

$$
S^{\mathrm{assort}}_{ij}(t)
=
\sum_{a \in [a_i,b_i]}
\sum_{c \in [a_j,b_j]}
w_i(a,t)\, w_j(c,t)\,
\frac{1}{a_{\mathrm{spread}}}
\exp\!\left(-\frac{|a-c|}{a_{\mathrm{spread}}}\right).
$$

Smaller values of $a_{\mathrm{spread}}$ correspond to more strongly age-assortative mixing.

### Parent–child mixing

Intergenerational mixing is informed by age-specific fertility patterns.  
Let $p_{\mathrm{pc}}(\Delta, y)$ denote the probability of a parent–child age gap $\Delta$ for a child born in year $y$. This quantity is computed by normalising the fertility rates by age, ensuring that the sum of the rates for a given year equals one.

For ages $a$ and $c$, we define the child’s age as

$$
a_{\mathrm{child}} = \min(a,c),
$$

with corresponding birth year $t - a_{\mathrm{child}}$. The parent–child contribution is then

$$
S^{\mathrm{pc}}_{ij}(t)
=
\sum_{a \in [a_i,b_i]}
\sum_{c \in [a_j,b_j]}
w_i(a,t)\, w_j(c,t)\,
p_{\mathrm{pc}}\!\left(|a-c|,\; t - a_{\mathrm{child}}\right).
$$

---

## Population scaling and asymmetry

The symmetric matrix $\mathbf{S}(t)$ represents contact intensity per pair of individuals.  
To obtain an asymmetric ``who-acquires-infection-from-whom'' matrix, each column is scaled by the population size of the contacted age group:

$$
C_{ij}(t) = S_{ij}(t)\, N_i(t),
$$

where $N_i(t)$ denotes the total population size of age group $i$ at time $t$.

---

## Normalisation

To ensure consistent overall contact intensity across time and parameter values, the matrix is normalised by its spectral radius. Let

$$
\rho(t) = \max \{ |\lambda| : \lambda \in \mathrm{eig}(\mathbf{C}(t)) \}.
$$

The final contact matrix used in the model is

$$
\tilde{\mathbf{C}}(t) = \frac{\mathbf{C}(t)}{\rho(t)}.
$$